In [ ]:
# !git clone https://github.com/airsplay/lxmert.git
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive/lxmert")

In [ ]:
#1. reformat train.json, val.json 
# !wget https://ivc.ischool.utexas.edu/VizWiz_final/vqa_data/Annotations.zip
# !unzip Annotations.zip
import json
with open('Annotations/train.json') as f1:
  train_json = json.load(f1)
with open('Annotations/val.json') as f2:
  val_json = json.load(f2)

answer_vocabs = set()

new_trains = []
for train in train_json:
  new_train = {}
  new_train["img_id"] = train["image"]
  new_train["question_id"] = train["image"]
  new_train["sent"] = train["question"]
  label_dict = {}
  for ans in train["answers"]:
    answer = ans["answer"]
    answer_vocabs.add(answer)
    label_dict[answer] = min(round(label_dict.get(answer,0) + 0.3, 1), 1.0)
  new_train["label"] = label_dict
  new_trains.append(new_train)
with open('data/vqa/train.json', 'w') as outfile:
    json.dump(new_trains, outfile)


new_vals = []
for val in val_json:
  new_val = {}
  new_val["img_id"] = val["image"]
  new_val["question_id"] = val["image"]
  new_val["sent"] = val["question"]
  label_dict = {}
  for ans in val["answers"]:
    answer = ans["answer"]
    answer_vocabs.add(answer)
    label_dict[answer] = min(round(label_dict.get(answer,0) + 0.3, 1), 1.0)
  new_val["label"] = label_dict
  new_vals.append(new_val)
with open('data/vqa/minival.json', 'w') as outfile:
    json.dump(new_vals, outfile)

print(len(new_trains))
print(len(new_vals))
print(len(answer_vocabs))

20523
4319
48730


In [ ]:
#2. generate ans2label and label2ans file
label2ans = list(answer_vocabs)
with open('data/vqa/trainval_label2ans.json', 'w') as outfile:
    json.dump(label2ans, outfile)
ans2label = {}
for i,ans in enumerate(label2ans):
  ans2label[ans] = i
with open('data/vqa/trainval_ans2label.json', 'w') as outfile:
    json.dump(ans2label, outfile)

!mkdir -p snap/pretrained 
!wget https://nlp.cs.unc.edu/data/model_LXRT.pth -P snap/pretrained

In [ ]:
!mkdir -p data/vizwiz_imgfeat
# !wget https://nlp.cs.unc.edu/data/lxmert_data/vizwiz/vizwiz/valid_obj36.tsv -P data/mscoco_imgfeat
# !wget 'https://s3.us-east-2.amazonaws.com/cmumonkey/36_vizwiz_val.pkl' -P data/vizwiz_imgfeat
!wget 'https://s3.us-east-2.amazonaws.com/cmumonkey/36_vizwiz_train.pkl' -P data/vizwiz_imgfeat

--2021-04-16 02:32:16--  https://s3.us-east-2.amazonaws.com/cmumonkey/36_vizwiz_train.pkl
Resolving s3.us-east-2.amazonaws.com (s3.us-east-2.amazonaws.com)... 52.219.96.106
Connecting to s3.us-east-2.amazonaws.com (s3.us-east-2.amazonaws.com)|52.219.96.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7122021558 (6.6G) [binary/octet-stream]
Saving to: ‘data/vizwiz_imgfeat/36_vizwiz_train.pkl’

36_vizwiz_train.pkl 100%[===================>]   6.63G  75.8MB/s    in 1m 53s  

2021-04-16 02:34:09 (60.2 MB/s) - ‘data/vizwiz_imgfeat/36_vizwiz_train.pkl’ saved [7122021558/7122021558]



In [ ]:
import pickle
with open('data/vizwiz_imgfeat/36_vizwiz_train.pkl', 'rb') as f:
    train_img_feat = pickle.load(f)
with open('data/vizwiz_imgfeat/36_vizwiz_val.pkl', 'rb') as f:
    val_img_feat = pickle.load(f)

KeyboardInterrupt: ignored

In [ ]:
print(val_img_feat[0].keys())
print(val_img_feat[0]['info'].item())

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 133kB 5.6MB/s 
     |████████████████████████████████| 7.4MB 14.9MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
ERROR: botocore 1.20.53 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
!bash run/vqa_finetune.bash 0 vizwiz_tiny --tiny

In [ ]:
!bash run/vqa_finetune.bash 0 vizwiz_all

In [ ]:
!bash run/vqa_test.bash 0 vizwiz_all_results --test valid --load snap/vqa/vizwiz_all/BEST

train
Load 20523 data from split(s) train.
Start to load Faster-RCNN detected objects from data/vizwiz_imgfeat/36_vizwiz_train.pkl
Loaded 512 images in file data/vizwiz_imgfeat/36_vizwiz_train.pkl in 98 seconds.
Use 444 data in torch dataset

LXRT encoder with 9 l_layers, 5 x_layers, and 5 r_layers.
BertAdam Total Iters: 52
Load model from snap/vqa/vizwiz_all/BEST
valid
Load 4319 data from split(s) valid.
Start to load Faster-RCNN detected objects from data/vizwiz_imgfeat/36_vizwiz_val.pkl
Loaded 7748 images in file data/vizwiz_imgfeat/36_vizwiz_val.pkl in 44 seconds.
Use 4317 data in torch dataset

0.4786889043317073


In [ ]:
import sys
import csv
import base64
import time
import pickle

import numpy as np
csv.field_size_limit(sys.maxsize)
FIELDNAMES = ["img_id", "img_h", "img_w", "objects_id", "objects_conf",
              "attrs_id", "attrs_conf", "num_boxes", "boxes", "features"]

def load_obj_pkl(fname, topk=None):
  data = []
  start_time = time.time()
  print("Start to load Faster-RCNN detected objects from %s" % fname)
  with open(fname,'rb') as f:
    img_feat = pickle.load(f)
    for i, item in enumerate(img_feat):
      item['img_h'] = int(item['image_h'])
      item['img_w'] = int(item['image_w'])
      item['num_boxes'] = int(item['num_bbox'])
      
      boxes = item['num_boxes']
      decode_config = [
              ('objects_id', (boxes, ), np.int64),
              ('objects_conf', (boxes, ), np.float32),
              ('attrs_id', (boxes, ), np.int64),
              ('attrs_conf', (boxes, ), np.float32)
          ]
      for key, shape, dtype in decode_config:
        # item[key] = np.frombuffer(base64.b64decode(item['info'].item().get(key)), dtype=dtype)
        item[key] = item['info'].item().get(key)
        item[key] = item[key].reshape(shape)
        item[key].setflags(write=False)
      
      # item['boxes'] = np.frombuffer(base64.b64decode(item['bbox']), dtype=np.float32)
      item['boxes'] = item['bbox']
      item['boxes'] = item['boxes'].reshape((boxes, 4))
      item['boxes'].setflags(write=False) 

      # item['features'] = np.frombuffer(base64.b64decode(item['x']), dtype=np.float32)
      item['features'] = item['x']
      item['features'] = item['features'].reshape((boxes, -1))
      item['features'].setflags(write=False) 

      item['img_id'] = item['info'].item().get('image_id') + ".jpg" 
      

      data.append(item['features'].shape[0])
      if topk is not None and len(data) == topk:
          break
    elapsed_time = time.time() - start_time
    print("Loaded %d images in file %s in %d seconds." % (len(data), fname, elapsed_time))
    return data


data = load_obj_pkl('data/vizwiz_imgfeat/36_vizwiz_train.pkl')


Start to load Faster-RCNN detected objects from data/vizwiz_imgfeat/36_vizwiz_train.pkl
Loaded 23952 images in file data/vizwiz_imgfeat/36_vizwiz_train.pkl in 44 seconds.


In [ ]:
from collections import Counter
print(Counter(data))
# {32, 34, 36, 21}

Counter({36: 23948, 21: 2, 34: 1, 32: 1})
